# Processing Large Climate Datasets with Dask and Xarray
### How to align memory chunks with on-disk chunks?
Efficient data processing on PAVICS, or using xarray/Dask in general, typically requires understanding two different notions of data *chunking*: 1) *on-disk* chunking or the way the data is stored in the filesystem and 2) *in-memory* chunking, or the way that Dask will break up a large dataset into manageable portions and process the data in parallel. In the case of *memory* chunks it is important to note that users can specify any size of in-memory chunking they feel is appropriate with respect to their system resources and the analysis at hand. However, this does not mean that the calculation will be efficient. Indeed, to efficiently process large datasets and prevent memory overloads, aligning in-memory chunks with the on-disk chunk structure is essential. This allows Dask to load and process data in a way that minimizes memory usage and I/O operations, speeding up computation.

I begin by checking the on-disk chunk structure of my dataset by loading it with `decode_times=False`. This skips time decoding and loads only metadata without reading the data into memory.

In [1]:
import xarray as xr 

# open the daily ERA5-land dataset as an example
url = "https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/datasets/reanalyses/day_ERA5-Land_NAM.ncml"

# use decode_times=False to skip time decoding and load only metadata
ds = xr.open_dataset(url, decode_times=False, chunks={})

# `._ChunkSizes` attribute shows the chunk structure of the variable
print(ds['tas']._ChunkSizes)

[366  50  50]


The `._ChunkSizes` attribute shows the chunk sizes of the `tas` variable on disk. I then re-open the dataset with matching Dask chunk sizes for efficient memory alignment. 

When you open a dataset using chunking, it is represented as Dask-backed arrays, where only the metadata is initially loaded, and the data itself remains on disk until needed. Dask uses lazy evalution, meaning it doesn't immediately perform the operation but instead builds a computation graph. This graph tracks the sequence of operations, delaying execution until `compute()` is explicitly called. During computation, Dask reads and processes data in chunks and loads only the necessary parts into memory. 

Here's an example showing the difference in computation times when the dataset is loaded with unaligned versus aligned chunking. For this example, I resample daily ERA5-Land `tas` data to yearly means. Since we are calculating a single year but for all of North America, a logical choice might be to chunk the data in memory for a few days over the `time` dimension, while keeping the entire spatial domain in the `lat` and `lon` dimensions:

In [2]:
# open dataset with unaligned chunks
ds = xr.open_dataset(url, chunks={"time":10, "lat":-1, "lon":-1})

# resample the 'tas' variable to yearly means
%time tas_resampled = ds['tas'].sel(time=slice('1981-01-01', '1981-12-31')).resample(time='YS').mean().compute()

CPU times: user 13.6 s, sys: 5.85 s, total: 19.5 s
Wall time: 5min 26s


However, this original chunking choice resulted in inefficient computation because the on-disk chunk size for the `time` dimension was much larger than 10. Processing such small chunks led to excessive overhead and slow performance. Instead of arbitrary chunking, I align the chunks with the on-disk structure by setting the `time` chunk size to match the smallest on-disk chunk and choosing spatial chunks in multiples of 50 to avoid creating too many small chunks:

In [4]:
# open dataset with aligned chunks
ds = xr.open_dataset(url, chunks={'time': 366, 'lat': 50*5, 'lon': 50*5})

# resample the 'tas' variable to yearly means
%time tas_resampled = ds['tas'].sel(time=slice('1981-01-01', '1981-12-31')).resample(time='YS').mean().compute()

CPU times: user 11.9 s, sys: 6.95 s, total: 18.8 s
Wall time: 1min 26s


The computation time dropped from approximately 5 minutes to just over 1 minute after aligning the chunks, as this approach better managed memory and I/O operations, leading to faster execution.

### How to do parallel processing with Dask Distributed Client? 
Processing large climate datasets can be computationally intensive and time-consuming due to their size and complexity. Parallel processing divides a computational task into smaller, independent subtasks that can be executed simultaneously across multiple processors or cores. This can minimize memory overhead and significantly reduce computation time. 

One way to implement parallel processing is through the Dask Distributed Client. When you initialize a Dask Client, you establish a connection to the cluster, allowing interaction with the scheduler and workers. The scheduler is the central component that manages task distribution across the workers, and the workers are the actual computational units that execute tasks using one or more CPU cores. 

Let's look at an example of resampling daily ERA5-Land data into yearly means for multiple variables using the Distributed Client. To manage system resources effectively, especially since PAVICS is a shared environment, I set the total memory limit to 20 GB. After experimenting with different configurations, I found that using 5 workers, with 2 threads per worker and 4 GB memory per worker provides a good balance between reducing overhead and optimizing computation time. I track memory usage and task progress in real time by displaying the Dask dashboard.

To create delayed tasks, I set `compute=False` with `to_zarr`. This delays the entire pipeline, from reading the data, performing computations, to writing the output, and allows Dask to optimize the execution plan. Dask schedules tasks efficiently, reducing unnecessary data movement and enabling each worker to write data independently to Zarr format. This approach is much faster than using sequential formats like NetCDF, which don’t support parallel writes as effectively.

To prevent memory leaks, which occur when the program retains memory it no longer needs, potentially slowing down the system or causing it to crash, I use the Client within a context manager to ensure that resources are released after execution. 

In [11]:
import xarray as xr
from pathlib import Path
from dask.distributed import Client
from dask import compute
import psutil  
import time

start_time = time.time()

# function to compute the yearly mean for each variable 
def compute_yearly_mean(ds, var_name):
    var = ds[var_name]
    yearly_mean = var.resample(time='YS').mean()
    return yearly_mean

# set up Dask client within a context manager
with Client(n_workers=5, threads_per_worker=2, memory_limit='4GB') as client:
    # display the Dask Dashboard
    display(client)

    # open the dataset with on-disk chunking structure
    url = "https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/datasets/reanalyses/day_ERA5-Land_NAM.ncml"
    ds = xr.open_dataset(url, chunks={'time': 366, 'lat': 50, 'lon':50}) 
    display(ds)

    # select data for the year 1981
    ds = ds.sel(time=slice('1981-01-01', '1981-12-31'))

    variables = list(ds.data_vars)

    # create a list to hold delayed tasks
    tasks = []
    for var_name in variables:
        output_path = Path(f"var_means/{var_name}_1981_yearly_mean.zarr")
        if not output_path.exists():
            yearly_mean = compute_yearly_mean(ds, var_name)
            # save to Zarr with compute=False to get a delayed task object
            delayed_task = yearly_mean.to_zarr(output_path, mode='w', compute=False)
            tasks.append(delayed_task)

    # trigger the execution of all delayed tasks
    compute(*tasks)

    # fetch memory usage from all workers and display the total usage
    worker_memory = client.run(lambda: psutil.Process().memory_info().rss)
    total_memory = sum(worker_memory.values())
    print(f"Total memory usage across all workers: {total_memory / 1e9:.2f} GB")

end_time = time.time()
elapsed_time = end_time - start_time
print(f'Total computation time: {elapsed_time:.2f} seconds')

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Workers: 5
Total threads: 10,Total memory: 18.63 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46341,Workers: 5
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Total threads: 10
Started: Just now,Total memory: 18.63 GiB
Comm: tcp://127.0.0.1:38031,Total threads: 2
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/36389/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:35377,


<xarray.Dataset>
Dimensions:  (lat: 800, lon: 1700, time: 26298)
Coordinates:
  * lat      (lat) float32 10.0 10.1 10.2 10.3 10.4 ... 89.5 89.6 89.7 89.8 89.9
  * lon      (lon) float32 -179.9 -179.8 -179.7 -179.6 ... -10.2 -10.1 -10.0
  * time     (time) datetime64[ns] 1950-01-01 1950-01-02 ... 2021-12-31
Data variables:
    tas      (time, lat, lon) float32 dask.array<chunksize=(13421, 50, 50), meta=np.ndarray>
    tasmin   (time, lat, lon) float32 dask.array<chunksize=(13421, 50, 50), meta=np.ndarray>
    tasmax   (time, lat, lon) float32 dask.array<chunksize=(13421, 50, 50), meta=np.ndarray>
    pr       (time, lat, lon) float32 dask.array<chunksize=(13421, 50, 50), meta=np.ndarray>
    prsn     (time, lat, lon) float32 dask.array<chunksize=(13421, 50, 50), meta=np.ndarray>
Attributes: (12/26)
    Conventions:          CF-1.8
    cell_methods:         time: mean (interval: 1 day)
    data_specs_version:   00.00.07
    domain:               NAM
    format:               netcdf
    frequency:            day
    ...                   ...
    dataset_description:  https://www.ecmwf.int/en/era5-land
    license_type:         permissive
    license:              Please acknowledge the use of ERA5-Land as stated i...
    attribution:          Contains modified Copernicus Climate Change Service...
    citation:             Muñoz Sabater, J., (2021): ERA5-Land hourly data fr...
    doi:                  https://doi.org/10.24381/cds.e2161bac

Total memory usage across all workers: 1.40 GB
Total computation time: 76.60 seconds


Let’s look at a more complex analysis involving heatwave indicators. Here, I calculate two climate indicators, `heat_wave_total_length` and `heat_wave_frequency`, using the `atmos` submodule from the `xclim` library. Both indicators rely on the same input data (`tasmin` and `tasmax`), so I create a pipeline of delayed tasks, which minimizes I/O operations by keeping the data in memory until both indicators are calculated and saved.

In this case, I chunk the `time` dimension into 4-year intervals to optimize memory usage, as we are analyzing a multi-year period (e.g., 1991–2010). An additional day is added to each 4-year chunk to account for leap years.

In [4]:
import time
from pathlib import Path

import psutil
import xarray as xr
from dask import compute
from dask.distributed import Client
from xclim import atmos

start_time = time.time()

with Client(n_workers=5, threads_per_worker=2, memory_limit="4GB") as client:
    display(client)

    # load data using on-disk chunk sizes
    url = "https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/datasets/reanalyses/day_ERA5-Land_NAM.ncml"
    ds = xr.open_dataset(url, chunks={"time": (365*4)+1, "lat": 50, "lon": 50})

    # we're interested in the 1991-2010 period for our heatwave analysis
    ds = ds.sel(time=slice("1991-01-01", "2010-12-31"))

    display(ds)

    # list of heatwave indicator functions
    indicators = [
        atmos.heat_wave_total_length,
        atmos.heat_wave_frequency,
        atmos._precip.precip_accumulation,
    ]
    tasks = []
    for indicator in indicators:
        ds_out = xr.Dataset(attrs=ds.attrs)  # create a new dataset for each indicator

        if indicator == atmos._precip.precip_accumulation:
            # calculate indicator
            out = indicator(
                ds=ds,
                freq="YS",
            )
        else:
            out = indicator(ds=ds, freq="YS", resample_before_rl=False)

        out = out.chunk({"time": -1, "lat": 50, "lon": 50})
        ds_out[out.name] = out

        output_path = Path(f"heatwave_output_ex1/{out.name}_1991-2010.zarr")
        output_path.parent.mkdir(parents=True, exist_ok=True)

        if not output_path.exists():
            # save to Zarr with compute=False to get a delayed task object
            delayed_task = ds_out.to_zarr(output_path, mode="w", compute=False)
            tasks.append(delayed_task)

    # trigger computation
    compute(*tasks)

    # fetch memory usage from all workers and display the total usage
    worker_memory = client.run(lambda: psutil.Process().memory_info().rss)
    total_memory = sum(worker_memory.values())
    print(f"Total memory usage across all workers: {total_memory / 1e9:.2f} GB")

end_time = time.time()
print(f"Total computation time: {end_time - start_time:.2f} seconds")

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Workers: 5
Total threads: 10,Total memory: 18.63 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38499,Workers: 5
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Total threads: 10
Started: Just now,Total memory: 18.63 GiB
Comm: tcp://127.0.0.1:38711,Total threads: 2
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/36357/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:39627,


<xarray.Dataset> Size: 199GB
Dimensions:  (lat: 800, lon: 1700, time: 7305)
Coordinates:
  * lat      (lat) float32 3kB 10.0 10.1 10.2 10.3 10.4 ... 89.6 89.7 89.8 89.9
  * lon      (lon) float32 7kB -179.9 -179.8 -179.7 -179.6 ... -10.2 -10.1 -10.0
  * time     (time) datetime64[ns] 58kB 1991-01-01 1991-01-02 ... 2010-12-31
Data variables:
    tas      (time, lat, lon) float32 40GB dask.array<chunksize=(1096, 50, 50), meta=np.ndarray>
    tasmin   (time, lat, lon) float32 40GB dask.array<chunksize=(1096, 50, 50), meta=np.ndarray>
    tasmax   (time, lat, lon) float32 40GB dask.array<chunksize=(1096, 50, 50), meta=np.ndarray>
    pr       (time, lat, lon) float32 40GB dask.array<chunksize=(1096, 50, 50), meta=np.ndarray>
    prsn     (time, lat, lon) float32 40GB dask.array<chunksize=(1096, 50, 50), meta=np.ndarray>
Attributes: (12/26)
    Conventions:          CF-1.8
    cell_methods:         time: mean (interval: 1 day)
    data_specs_version:   00.00.07
    domain:               NAM
    format:               netcdf
    frequency:            day
    ...                   ...
    dataset_description:  https://www.ecmwf.int/en/era5-land
    license_type:         permissive
    license:              Please acknowledge the use of ERA5-Land as stated i...
    attribution:          Contains modified Copernicus Climate Change Service...
    citation:             Muñoz Sabater, J., (2021): ERA5-Land hourly data fr...
    doi:                  https://doi.org/10.24381/cds.e2161bac

/opt/conda/envs/birdy/lib/python3.11/site-packages/xclim/core/cfchecks.py:43: UserWarning: Variable has a non-conforming cell_methods: Got `time: point`, which do not include the expected `time: minimum`.
  _check_cell_methods(
/opt/conda/envs/birdy/lib/python3.11/site-packages/xclim/core/cfchecks.py:43: UserWarning: Variable has a non-conforming cell_methods: Got `time: point`, which do not include the expected `time: maximum`.
  _check_cell_methods(
/opt/conda/envs/birdy/lib/python3.11/site-packages/xclim/core/cfchecks.py:43: UserWarning: Variable has a non-conforming cell_methods: Got `time: point`, which do not include the expected `time: minimum`.
  _check_cell_methods(
/opt/conda/envs/birdy/lib/python3.11/site-packages/xclim/core/cfchecks.py:43: UserWarning: Variable has a non-conforming cell_methods: Got `time: point`, which do not include the expected `time: maximum`.
  _check_cell_methods(


Total memory usage across all workers: 2.14 GB
Total computation time: 1608.42 seconds


### What can we do when we have large task graphs / large memory footprint? 
One downside of using a fully delayed computation approach is that it can lead to the creation of large task graphs that are difficult to manage. This can result in excessive memory consumption as the Dask scheduler struggles to handle numerous interdependent tasks.

To address this issue, we can simplify the task graph by computing and saving each indicator one at a time. This method ensures that Dask completes the calculation and writing of the first indicator, then releases the memory used for that computation before moving on to the second indicator. By processing each indicator sequentially, the memory footprint is reduced, and the scheduler has fewer tasks to manage at any given time.

In [5]:
from pathlib import Path
from dask.distributed import Client
import time
from xclim import atmos
from dask import compute
import psutil  

start_time = time.time()

with Client(n_workers=5, threads_per_worker=2, memory_limit='4GB') as client:
    display(client)
    
    url = "https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/datasets/reanalyses/day_ERA5-Land_NAM.ncml"
    ds = xr.open_dataset(url, chunks={'time': (365*4)+1, 'lat': 50, 'lon': 50})
    
    ds = ds.sel(time=slice('1991-01-01', '2010-12-31'))
    
    display(ds)

    indicators = [atmos.heat_wave_total_length, atmos.heat_wave_frequency, atmos._precip.precip_accumulation]  

    for indicator in indicators:
        ds_out = xr.Dataset(attrs=ds.attrs) 
        
        if indicator == atmos._precip.precip_accumulation:
            # calculate indicator
            out = indicator(
                ds=ds,
                freq='YS',
            )
        else: 
            out = indicator(
                ds=ds,
                freq='YS',
                resample_before_rl=False
            )
            
        out = out.chunk({'time': -1, 'lat': 50, 'lon': 50})
        ds_out[out.name] = out

        output_path = Path(f'heatwave_output_ex2/{out.name}_1991-2010.zarr')
        output_path.parent.mkdir(parents=True, exist_ok=True)

        if not output_path.exists():
            # save to Zarr, triggering computation immediately
            ds_out.to_zarr(output_path)
    
    # fetch memory usage from all workers and display the total usage
    worker_memory = client.run(lambda: psutil.Process().memory_info().rss)
    total_memory = sum(worker_memory.values())
    print(f"Total memory usage across all workers: {total_memory / 1e9:.2f} GB")

end_time = time.time()
print(f"Total computation time: {end_time - start_time:.2f} seconds")

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Workers: 5
Total threads: 10,Total memory: 18.63 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34229,Workers: 5
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Total threads: 10
Started: Just now,Total memory: 18.63 GiB
Comm: tcp://127.0.0.1:34223,Total threads: 2
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/40455/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:39307,


<xarray.Dataset> Size: 199GB
Dimensions:  (lat: 800, lon: 1700, time: 7305)
Coordinates:
  * lat      (lat) float32 3kB 10.0 10.1 10.2 10.3 10.4 ... 89.6 89.7 89.8 89.9
  * lon      (lon) float32 7kB -179.9 -179.8 -179.7 -179.6 ... -10.2 -10.1 -10.0
  * time     (time) datetime64[ns] 58kB 1991-01-01 1991-01-02 ... 2010-12-31
Data variables:
    tas      (time, lat, lon) float32 40GB dask.array<chunksize=(1096, 50, 50), meta=np.ndarray>
    tasmin   (time, lat, lon) float32 40GB dask.array<chunksize=(1096, 50, 50), meta=np.ndarray>
    tasmax   (time, lat, lon) float32 40GB dask.array<chunksize=(1096, 50, 50), meta=np.ndarray>
    pr       (time, lat, lon) float32 40GB dask.array<chunksize=(1096, 50, 50), meta=np.ndarray>
    prsn     (time, lat, lon) float32 40GB dask.array<chunksize=(1096, 50, 50), meta=np.ndarray>
Attributes: (12/26)
    Conventions:          CF-1.8
    cell_methods:         time: mean (interval: 1 day)
    data_specs_version:   00.00.07
    domain:               NAM
    format:               netcdf
    frequency:            day
    ...                   ...
    dataset_description:  https://www.ecmwf.int/en/era5-land
    license_type:         permissive
    license:              Please acknowledge the use of ERA5-Land as stated i...
    attribution:          Contains modified Copernicus Climate Change Service...
    citation:             Muñoz Sabater, J., (2021): ERA5-Land hourly data fr...
    doi:                  https://doi.org/10.24381/cds.e2161bac

/opt/conda/envs/birdy/lib/python3.11/site-packages/xclim/core/cfchecks.py:43: UserWarning: Variable has a non-conforming cell_methods: Got `time: point`, which do not include the expected `time: minimum`.
  _check_cell_methods(
/opt/conda/envs/birdy/lib/python3.11/site-packages/xclim/core/cfchecks.py:43: UserWarning: Variable has a non-conforming cell_methods: Got `time: point`, which do not include the expected `time: maximum`.
  _check_cell_methods(
/opt/conda/envs/birdy/lib/python3.11/site-packages/xclim/core/cfchecks.py:43: UserWarning: Variable has a non-conforming cell_methods: Got `time: point`, which do not include the expected `time: minimum`.
  _check_cell_methods(
/opt/conda/envs/birdy/lib/python3.11/site-packages/xclim/core/cfchecks.py:43: UserWarning: Variable has a non-conforming cell_methods: Got `time: point`, which do not include the expected `time: maximum`.
  _check_cell_methods(


Total memory usage across all workers: 1.95 GB
Total computation time: 2820.25 seconds


Even with the sequential computation approach, there may be scenarios where the heatwave indicator data is still too large to write in a single step. In such cases, an effective strategy is to split the dataset into smaller, manageable spatial chunks, which would allow for more efficient processing and data writing.

For example, we can calculate the heatwave indicators over the entire dataset spanning North America, and then divide the results into smaller latitude bins (e.g., groups of 50 latitudes each). Each bin can then be processed and saved individually to a temporary `.zarr` file using Dask. Once all bins are saved, they can be merged back into a single dataset and written out as a final `.zarr` file. This method distributes the data processing load and helps minimize the strain on system resources.

In [6]:
import xarray as xr
from xclim import atmos
from pathlib import Path
from xscen.io import save_to_zarr
from tempfile import TemporaryDirectory
from dask.distributed import Client
import shutil
import time

start_time = time.time()

url = "https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/datasets/reanalyses/day_ERA5-Land_NAM.ncml"
ds = xr.open_dataset(url, chunks={'time': (365*4)+1, 'lat': 50, 'lon': 50})

# we're interested in the 1991-2010 period for our heatwave analysis
ds = ds.sel(time=slice('1991-01-01', '2010-12-31'))

display(ds)

# create an empty output dataset
dsout = xr.Dataset(attrs=ds.attrs)

# xclim calculation on entire dataset
out = atmos.heat_wave_total_length(ds=ds, freq='YS', resample_before_rl=False) # Heat_wave calculation where heat-waves could cross calendar years
dsout[out.name] = out

outzarr = Path('output').joinpath('heat_wave_total_length_spatial_example.zarr')

if not outzarr.exists():
    
    # if data is too big to write in single step - make individual datasets by binning the `lat` dim (approx n= 50 latitudes at a time)
    grp_dim     = 'lat'
    bins        = round(len(dsout.lat)/50) 
    _, datasets = zip(*dsout.groupby_bins(grp_dim, bins))
    print(len(datasets))
    assert sum([len(d[grp_dim]) for d in datasets]) == len(dsout[grp_dim])
    
    # export each chunk of 50-latitudes to a temporary location
    with TemporaryDirectory(prefix='output', dir='.') as outtmp:
        for ii, dds in enumerate(datasets):
            dds = dds.chunk(time=-1, lon=50, lat=50)
            filename = Path(outtmp).joinpath(f"{ii}.zarr")
            with Client(n_workers=5, threads_per_worker=2, memory_limit='4GB') as client: 
                display(client)
                save_to_zarr(
                    ds=dds,
                    filename=filename,
                    mode='o',
                )
        
        # reassamble pieces and export joined
        inzarrs = sorted(list(filename.parent.glob(f'*.zarr')))
        
        # open the files as a combined multi-file dataset
        ds = xr.open_mfdataset(inzarrs, engine='zarr', decode_timedelta=False)  
        
        # define the final chunking configuration
        final_chunks = dict(time=-1, lon=50, lat=50) 
        
        # save the final combined dataset
        tmpzarr = Path(outtmp).joinpath(outzarr.name)
        with Client(n_workers=10) as c:
            display(client)
            save_to_zarr(
                    ds=ds.chunk(final_chunks),
                    filename=tmpzarr,
                    mode="o",
                )
        # move the final combined file to the output location
        outzarr.parent.mkdir(exist_ok=True, parents=True)
        shutil.move(tmpzarr, outzarr)
        
end_time = time.time()
print(f"Total computation time: {end_time - start_time:.2f} seconds")

<xarray.Dataset> Size: 199GB
Dimensions:  (lat: 800, lon: 1700, time: 7305)
Coordinates:
  * lat      (lat) float32 3kB 10.0 10.1 10.2 10.3 10.4 ... 89.6 89.7 89.8 89.9
  * lon      (lon) float32 7kB -179.9 -179.8 -179.7 -179.6 ... -10.2 -10.1 -10.0
  * time     (time) datetime64[ns] 58kB 1991-01-01 1991-01-02 ... 2010-12-31
Data variables:
    tas      (time, lat, lon) float32 40GB dask.array<chunksize=(1096, 50, 50), meta=np.ndarray>
    tasmin   (time, lat, lon) float32 40GB dask.array<chunksize=(1096, 50, 50), meta=np.ndarray>
    tasmax   (time, lat, lon) float32 40GB dask.array<chunksize=(1096, 50, 50), meta=np.ndarray>
    pr       (time, lat, lon) float32 40GB dask.array<chunksize=(1096, 50, 50), meta=np.ndarray>
    prsn     (time, lat, lon) float32 40GB dask.array<chunksize=(1096, 50, 50), meta=np.ndarray>
Attributes: (12/26)
    Conventions:          CF-1.8
    cell_methods:         time: mean (interval: 1 day)
    data_specs_version:   00.00.07
    domain:               NAM
    format:               netcdf
    frequency:            day
    ...                   ...
    dataset_description:  https://www.ecmwf.int/en/era5-land
    license_type:         permissive
    license:              Please acknowledge the use of ERA5-Land as stated i...
    attribution:          Contains modified Copernicus Climate Change Service...
    citation:             Muñoz Sabater, J., (2021): ERA5-Land hourly data fr...
    doi:                  https://doi.org/10.24381/cds.e2161bac

/opt/conda/envs/birdy/lib/python3.11/site-packages/xclim/core/cfchecks.py:43: UserWarning: Variable has a non-conforming cell_methods: Got `time: point`, which do not include the expected `time: minimum`.
/opt/conda/envs/birdy/lib/python3.11/site-packages/xclim/core/cfchecks.py:43: UserWarning: Variable has a non-conforming cell_methods: Got `time: point`, which do not include the expected `time: maximum`.


16


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Workers: 5
Total threads: 10,Total memory: 18.63 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38343,Workers: 5
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Total threads: 10
Started: Just now,Total memory: 18.63 GiB
Comm: tcp://127.0.0.1:36271,Total threads: 2
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/44179/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:37383,


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Workers: 5
Total threads: 10,Total memory: 18.63 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46327,Workers: 5
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Total threads: 10
Started: Just now,Total memory: 18.63 GiB
Comm: tcp://127.0.0.1:44655,Total threads: 2
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/37769/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:41403,


...

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Workers: 5
Total threads: 10,Total memory: 18.63 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46883,Workers: 5
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Total threads: 10
Started: Just now,Total memory: 18.63 GiB
Comm: tcp://127.0.0.1:35293,Total threads: 2
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/44661/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:38377,


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Workers: 5
Total threads: 10,Total memory: 18.63 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34863,Workers: 5
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Total threads: 10
Started: Just now,Total memory: 18.63 GiB
Comm: tcp://127.0.0.1:38911,Total threads: 2
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/37073/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:35375,


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Workers: 5
Total threads: 10,Total memory: 18.63 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34111,Workers: 5
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Total threads: 10
Started: Just now,Total memory: 18.63 GiB
Comm: tcp://127.0.0.1:42335,Total threads: 2
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/43955/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:35723,


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Status: closed,Using processes: True
Comm: tcp://127.0.0.1:34111,Workers: 0
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Total threads: 0
Started: 1 minute ago,Total memory: 0 B


Total computation time: 1173.29 seconds


Alternatively, if our goal is to calculate heatwave indicators over a longer time period but for a smaller spatial extent, we can manage memory more efficiently by splitting the data along the time dimension, such as into 5-year intervals.

In [7]:
import xarray as xr
from xclim import atmos
from pathlib import Path
from xscen.io import save_to_zarr
from tempfile import TemporaryDirectory
from dask.distributed import Client
import shutil
import pandas as pd
import time

start_time = time.time()

url = "https://pavics.ouranos.ca/twitcher/ows/proxy/thredds/dodsC/datasets/reanalyses/day_ERA5-Land_NAM.ncml"
ds = xr.open_dataset(url, chunks={'time': (365*4)+1, 'lat': 50, 'lon': 50})

# this time, let's focus our analysis on a smaller region and a longer time period  
ds = ds.sel(time=slice('1950-01-01', '2020-12-31'), lat=slice(42.1, 65.0), lon=(slice(-80.0, -73.5)))

display(ds)

dsout = xr.Dataset(attrs=ds.attrs)

out = atmos.heat_wave_total_length(ds=ds, freq='YS', resample_before_rl=False)
dsout[out.name] = out

outzarr = Path('output').joinpath('heat_wave_total_length_temporal_example.zarr')

if not outzarr.exists():
    
    # resample into 5-year intervals
    _, datasets = zip(*dsout.resample(time='5YS'))

    # export each 5-year chunk to a temporary location
    with TemporaryDirectory(prefix='output', dir='.') as outtmp:
        for ii, dds in enumerate(datasets):
            dds = dds.chunk(dict(time=-1, lon=50, lat=50))  
            filename = Path(outtmp).joinpath(f"{ii}.zarr")
            with Client(n_workers=5, threads_per_worker=2, memory_limit='4GB') as client: 
                display(client)
                save_to_zarr(
                    ds=dds,
                    filename=filename,
                    mode='o',
                )
        
        # reassemble the 5-year chunks into a single dataset
        inzarrs = sorted(list(filename.parent.glob(f'*.zarr')))
        
        # open the files as a combined multi-file dataset
        ds = xr.open_mfdataset(inzarrs, engine='zarr', decode_timedelta=False)
        
        # define the final chunking configuration
        final_chunks = dict(time=12*50, lon=50, lat=50)
        
        # save the final combined dataset
        tmpzarr = Path(outtmp).joinpath(outzarr.name)
        with Client(n_workers=10) as c:
            display(client)
            save_to_zarr(
                ds=ds.chunk(final_chunks),
                filename=tmpzarr,
                mode="o",
            )
            
        # move the final combined file to the output location
        outzarr.parent.mkdir(exist_ok=True, parents=True)
        shutil.move(tmpzarr, outzarr)
        
end_time = time.time()
print(f"Total computation time: {end_time - start_time:.2f} seconds")

<xarray.Dataset> Size: 8GB
Dimensions:  (lat: 230, lon: 66, time: 25933)
Coordinates:
  * lat      (lat) float32 920B 42.1 42.2 42.3 42.4 42.5 ... 64.7 64.8 64.9 65.0
  * lon      (lon) float32 264B -80.0 -79.9 -79.8 -79.7 ... -73.7 -73.6 -73.5
  * time     (time) datetime64[ns] 207kB 1950-01-01 1950-01-02 ... 2020-12-31
Data variables:
    tas      (time, lat, lon) float32 2GB dask.array<chunksize=(1461, 29, 1), meta=np.ndarray>
    tasmin   (time, lat, lon) float32 2GB dask.array<chunksize=(1461, 29, 1), meta=np.ndarray>
    tasmax   (time, lat, lon) float32 2GB dask.array<chunksize=(1461, 29, 1), meta=np.ndarray>
    pr       (time, lat, lon) float32 2GB dask.array<chunksize=(1461, 29, 1), meta=np.ndarray>
    prsn     (time, lat, lon) float32 2GB dask.array<chunksize=(1461, 29, 1), meta=np.ndarray>
Attributes: (12/26)
    Conventions:          CF-1.8
    cell_methods:         time: mean (interval: 1 day)
    data_specs_version:   00.00.07
    domain:               NAM
    format:               netcdf
    frequency:            day
    ...                   ...
    dataset_description:  https://www.ecmwf.int/en/era5-land
    license_type:         permissive
    license:              Please acknowledge the use of ERA5-Land as stated i...
    attribution:          Contains modified Copernicus Climate Change Service...
    citation:             Muñoz Sabater, J., (2021): ERA5-Land hourly data fr...
    doi:                  https://doi.org/10.24381/cds.e2161bac

/opt/conda/envs/birdy/lib/python3.11/site-packages/xclim/core/cfchecks.py:43: UserWarning: Variable has a non-conforming cell_methods: Got `time: point`, which do not include the expected `time: minimum`.
/opt/conda/envs/birdy/lib/python3.11/site-packages/xclim/core/cfchecks.py:43: UserWarning: Variable has a non-conforming cell_methods: Got `time: point`, which do not include the expected `time: maximum`.


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Workers: 5
Total threads: 10,Total memory: 18.63 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41185,Workers: 5
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Total threads: 10
Started: Just now,Total memory: 18.63 GiB
Comm: tcp://127.0.0.1:34781,Total threads: 2
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/43149/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:40091,


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Workers: 5
Total threads: 10,Total memory: 18.63 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33175,Workers: 5
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Total threads: 10
Started: Just now,Total memory: 18.63 GiB
Comm: tcp://127.0.0.1:41697,Total threads: 2
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/39473/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:44297,


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Workers: 5
Total threads: 10,Total memory: 18.63 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:41255,Workers: 5
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Total threads: 10
Started: Just now,Total memory: 18.63 GiB
Comm: tcp://127.0.0.1:35735,Total threads: 2
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/33573/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:46743,


...

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Workers: 5
Total threads: 10,Total memory: 18.63 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33637,Workers: 5
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Total threads: 10
Started: Just now,Total memory: 18.63 GiB
Comm: tcp://127.0.0.1:40011,Total threads: 2
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/43153/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:46557,


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Workers: 5
Total threads: 10,Total memory: 18.63 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44877,Workers: 5
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Total threads: 10
Started: Just now,Total memory: 18.63 GiB
Comm: tcp://127.0.0.1:32957,Total threads: 2
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/36011/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:33431,


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Workers: 5
Total threads: 10,Total memory: 18.63 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46781,Workers: 5
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Total threads: 10
Started: Just now,Total memory: 18.63 GiB
Comm: tcp://127.0.0.1:35469,Total threads: 2
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/33693/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:44343,


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Status: closed,Using processes: True
Comm: tcp://127.0.0.1:46781,Workers: 0
Dashboard: https://pavics.ouranos.ca/jupyter/user/aslibese/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


Total computation time: 667.90 seconds
